In [1]:
import psycopg2
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import grequests

/usr/local/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


## Data Sources

### Postgres Database

In [2]:
conn_string = "host='db.lisbondatascience.org' \
               dbname='datawrangling' \
               user='ldsa_student' \
               password='Z7P3FpkejYgH6jkN'"
conn = psycopg2.connect(conn_string)

In [3]:
query = 'SELECT * FROM hackathon.credit_risk_target WHERE "TARGET" IS NOT null;'
train_id_df = pd.read_sql_query(query, con=conn)

In [4]:
train_id_df.head()

,SK_ID_CURR,TARGET
0,100006,0
1,100014,0
2,100015,0
3,100017,0
4,100018,0


In [52]:
train_id_df.to_csv('data/train_ids.csv')

In [5]:
query = 'SELECT * FROM hackathon.credit_risk_target WHERE "TARGET" IS null;'
test_id_df = pd.read_sql_query(query, con=conn)

In [6]:
test_id_df.head()

,SK_ID_CURR,TARGET
0,100004,None
1,100012,None
2,100034,None
3,100043,None
4,100055,None


In [53]:
test_id_df.to_csv('data/test_ids.csv')

In [7]:
query = 'SELECT * FROM hackathon.credit_risk_features;'
postgres_df = pd.read_sql_query(query, con=conn)

In [8]:
postgres_df.head()

,SK_ID_CURR,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,BASEMENTAREA_AVG,ENTRANCES_AVG,FLAG_DOCUMENT_20,FONDKAPREMONT_MODE,CNT_CHILDREN,FLAG_DOCUMENT_15,FLAG_EMAIL,OWN_CAR_AGE
0,100004,0.0,-815.0,NaN,NaN,0,None,0,0,0,26.0
1,100006,0.0,-617.0,NaN,NaN,0,None,0,0,0,NaN
2,100012,0.0,-1673.0,NaN,NaN,0,None,0,0,0,NaN
3,100014,0.0,-844.0,NaN,NaN,0,None,1,0,0,NaN
4,100015,0.0,-2396.0,NaN,NaN,0,None,0,0,0,NaN


In [54]:
postgres_df.to_csv('data/credit_risk_features.csv')

### API

In [9]:
import requests

In [10]:
API_BASE = 'http://35.180.129.212:8001'

In [11]:
res = requests.get(API_BASE + '/api/token-auth?username=smwade')
data = res.json()
API_TOKEN = data['token']
print('API_TOKEN: {}'.format(API_TOKEN))

API_TOKEN: 0bee8cd0757a7279cffdba595a3055a5183f135c


In [ ]:
api_dataset = []
for i in tqdm(range(1, 51)):
    res = requests.get(API_BASE + '/api/data?page={}&page_size=2000'.format(i),
                   headers={'Authorization': 'Token ' + API_TOKEN})
    data = res.json()
    api_dataset += data['results']

In [ ]:
api_df = pd.DataFrame(api_dataset)

In [ ]:
api_df.to_csv('api_page.csv')

In [ ]:
api_df.head()

```json
{
"sk_id_curr": 100004,
"flag_document_12": false,
"amt_req_credit_bureau_qrt": 0,
"name_education_type": "Secondary / secondary special",
"name_income_type": "Working",
"flag_document_13": false,
"emergencystate_mode": "",
"days_birth": -19046,
"apartments_avg": null,
"flag_document_9": false,
"obs_30_cnt_social_circle": 0
}
```

In [13]:
all_ids = list(test_id_df['SK_ID_CURR']) + list(train_id_df['SK_ID_CURR'])

In [ ]:
def exception_handler(request, exception):
    print(request)

urls = [API_BASE + '/api/data/{}'.format(str(x)) for x in all_ids]

rs = (grequests.get(u, headers={'Authorization': 'Token ' + API_TOKEN}, timeout=1) for u in urls)

res_list = grequests.map(rs, exception_handler=exception_handler)

In [ ]:
new_data = [x.json() for x in res_list]
df_api_read = pd.DataFrame(new_data)